# 04. Memory Agent (Short-term Memory)

간단한 **대화 히스토리**(메모리)를 유지하여 문맥을 반영하는 에이전트를 만듭니다.


In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import AzureOpenAI

# Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview"
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
class MemoryAgent:
    def __init__(self, model: str = None, max_turns: int = 6):
        # Azure OpenAI 배포명 사용
        self.model = model or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
        self.max_turns = max_turns
        self.history = [{"role":"system","content":"You are a helpful assistant with short-term memory."}]
    
    def ask(self, user_text: str) -> str:
        self.history.append({"role":"user","content": user_text})
        # 최근 대화만 유지 (토큰 절약)
        context = [self.history[0]] + self.history[-(self.max_turns+1):]
        completion = client.chat.completions.create(
            model=self.model,
            messages=context,
            max_tokens=200,
            temperature=0.7
        )
        reply = completion.choices[0].message.content.strip()
        self.history.append({"role":"assistant","content": reply})
        return reply

## Try it

In [ ]:
agent = MemoryAgent()
print(agent.ask("내 영어 말하기를 2주 안에 올리고 싶은데, 데일리 루틴 만들어줘."))
print(agent.ask("좋아. 그 루틴을 평일/주말로 나눠서 다시 정리해줘."))